In [62]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [63]:
import os

In [64]:
# os.makedirs('train_img_1')
# os.makedirs('test_img_1')

In [65]:
import torch
from torch import nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn

In [66]:
df=pd.read_csv("/kaggle/input/amazon-mll/amazon_ml/train.csv")

In [67]:
df.head()

,image_link,group_id,entity_name,entity_value
0,https://m.media-amazon.com/images/I/61I9XdN6OF...,748919,item_weight,500.0 gram
1,https://m.media-amazon.com/images/I/71gSRbyXmo...,916768,item_volume,1.0 cup
2,https://m.media-amazon.com/images/I/61BZ4zrjZX...,459516,item_weight,0.709 gram
3,https://m.media-amazon.com/images/I/612mrlqiI4...,459516,item_weight,0.709 gram
4,https://m.media-amazon.com/images/I/617Tl40LOX...,731432,item_weight,1400 milligram


In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 263859 entries, 0 to 263858
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   image_link    263859 non-null  object
 1   group_id      263859 non-null  int64 
 2   entity_name   263859 non-null  object
 3   entity_value  263859 non-null  object
dtypes: int64(1), object(3)
memory usage: 8.1+ MB


In [69]:
df['entity_name'].unique()

array(['item_weight', 'item_volume', 'voltage', 'wattage',
       'maximum_weight_recommendation', 'height', 'depth', 'width'],
      dtype=object)

In [70]:
maxlen_entity_value=max([len(tex) for tex in df['entity_value'].unique()])

In [71]:
maxlen_entity_value

32

In [72]:
df['image_link'][0]

'https://m.media-amazon.com/images/I/61I9XdN6OFL.jpg'

In [73]:
test_df=pd.read_csv("/kaggle/input/amll-1/test.csv")

In [74]:
test_df.head()

,index,image_link,group_id,entity_name
0,0,https://m.media-amazon.com/images/I/110EibNycl...,156839,height
1,1,https://m.media-amazon.com/images/I/11TU2clswz...,792578,width
2,2,https://m.media-amazon.com/images/I/11TU2clswz...,792578,height
3,3,https://m.media-amazon.com/images/I/11TU2clswz...,792578,depth
4,4,https://m.media-amazon.com/images/I/11gHj8dhhr...,792578,depth


In [75]:
entity_unit_map = {
    'width': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'depth': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'height': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'item_weight': {'gram',
        'kilogram',
        'microgram',
        'milligram',
        'ounce',
        'pound',
        'ton'},
    'maximum_weight_recommendation': {'gram',
        'kilogram',
        'microgram',
        'milligram',
        'ounce',
        'pound',
        'ton'},
    'voltage': {'kilovolt', 'millivolt', 'volt'},
    'wattage': {'kilowatt', 'watt'},
    'item_volume': {'centilitre',
        'cubic foot',
        'cubic inch',
        'cup',
        'decilitre',
        'fluid ounce',
        'gallon',
        'imperial gallon',
        'litre',
        'microlitre',
        'millilitre',
        'pint',
        'quart'}
}

allowed_units = {unit for entity in entity_unit_map for unit in entity_unit_map[entity]}

In [76]:
import re
import os
import requests
import pandas as pd
import multiprocessing
import time
from time import time as timer
from tqdm import tqdm
import numpy as np
from pathlib import Path
from functools import partial
import requests
import urllib
from PIL import Image

def common_mistake(unit):
    if unit in allowed_units:
        return unit
    if unit.replace('ter', 'tre') in allowed_units:
        return unit.replace('ter', 'tre')
    if unit.replace('feet', 'foot') in allowed_units:
        return unit.replace('feet', 'foot')
    return unit

def parse_string(s):
    s_stripped = "" if s==None or str(s)=='nan' else s.strip()
    if s_stripped == "":
        return None, None
    pattern = re.compile(r'^-?\d+(\.\d+)?\s+[a-zA-Z\s]+$')
    if not pattern.match(s_stripped):
        raise ValueError("Invalid format in {}".format(s))
    parts = s_stripped.split(maxsplit=1)
    number = float(parts[0])
    unit = common_mistake(parts[1])
    if unit not in allowed_units:
        raise ValueError("Invalid unit [{}] found in {}. Allowed units: {}".format(
            unit, s, allowed_units))
    return number, unit


def create_placeholder_image(image_save_path):
    try:
        placeholder_image = Image.new('RGB', (100, 100), color='black')
        placeholder_image.save(image_save_path)
    except Exception as e:
        return

def download_image(image_link, save_folder, retries=3, delay=3):
    if not isinstance(image_link, str):
        return

    filename = Path(image_link).name
    image_save_path = os.path.join(save_folder, filename)

    if os.path.exists(image_save_path):
        return

    for _ in range(retries):
        try:
            urllib.request.urlretrieve(image_link, image_save_path)
            return
        except:
            time.sleep(delay)

    create_placeholder_image(image_save_path) #Create a black placeholder image for invalid links/images

def download_images(image_links, download_folder, allow_multiprocessing=True):
    if not os.path.exists(download_folder):
        os.makedirs(download_folder)

    if allow_multiprocessing:
        download_image_partial = partial(
            download_image, save_folder=download_folder, retries=3, delay=3)

        with multiprocessing.Pool() as pool:
            list(tqdm(pool.imap(download_image_partial, image_links), total=len(image_links)))
            pool.close()
            pool.join()
    else:
        for image_link in tqdm(image_links, total=len(image_links)):
            download_image(image_link, save_folder=download_folder, retries=3, delay=3)


In [77]:
import concurrent.futures

In [78]:
# with concurrent.futures.ThreadPoolExecutor(max_workers=10) as excutor:
#     submitted = list(excutor.submit(download_image,img_link,'train_img_1') for img_link in df['image_link'].values)
#     for futue_value in tqdm(concurrent.futures.as_completed(submitted)):
#         _ = futue_value.result()


In [79]:
test_df.shape

(131187, 4)

In [80]:
# for img_link in tqdm(test_df['image_link'].values):
#     download_image(img_link,'test_img_1')

In [81]:
#download_images(df['image_link'],'train_img_1',allow_multiprocessing=False)

In [82]:
#download_images(test_df['image_link'],'test_img_1')

In [83]:
df['image_link'][len(df['image_link'])-10:]

263849    https://m.media-amazon.com/images/I/61-1ebQE-9...
263850    https://m.media-amazon.com/images/I/51xvxFlRXo...
263851    https://m.media-amazon.com/images/I/51glSG-nd+...
263852    https://m.media-amazon.com/images/I/51-tbcJ4qb...
263853    https://m.media-amazon.com/images/I/515WWrQ0UI...
263854    https://m.media-amazon.com/images/I/612J1R1xHl...
263855    https://m.media-amazon.com/images/I/61Blzh2+28...
263856    https://m.media-amazon.com/images/I/51MsegDL9V...
263857    https://m.media-amazon.com/images/I/510KhVw4VS...
263858    https://m.media-amazon.com/images/I/51lzTNLQ-6...
Name: image_link, dtype: object

In [84]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [85]:
train_dir='/kaggle/input/amazon-mll/amazon_ml/train_img_1'
#test_dir='D:/amazon_ml/test_img_1'

In [86]:
import torch
import PIL
from torch.utils.data import DataLoader
from torchvision import transforms

In [87]:
!pip install Pillow
from PIL import ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES = True

^C
ERROR: Operation cancelled by user


In [88]:
df.head(3)

,image_link,group_id,entity_name,entity_value
0,https://m.media-amazon.com/images/I/61I9XdN6OF...,748919,item_weight,500.0 gram
1,https://m.media-amazon.com/images/I/71gSRbyXmo...,916768,item_volume,1.0 cup
2,https://m.media-amazon.com/images/I/61BZ4zrjZX...,459516,item_weight,0.709 gram


In [89]:
class CustomDataset(torch.utils.data.dataset.Dataset):
    def __init__(self,dataset,transform,source_dir,target_length,entity_transform,target_encoder):
        self.dataset = dataset
        self.transform = transform
        self.source_dir = source_dir
        self.entity_transform = entity_transform
        self.target_encoder= target_encoder
        self.target_length = target_length

    def __len__(self):
        return self.dataset.shape[0]

    def __get_image(self,path):
        img_array = PIL.Image.open(path)
        transformed_img = self.transform(img_array)
        #print(f'transimage: {transformed_img.shape}')
        if transformed_img.shape[0]==1:
            transformed_img = transformed_img.repeat(3,1,1)
        return transformed_img

    def __create_target(self,target):
        char_lvl = ['<sos>']+list(s for s in target)+['<eos>']
        extra_length = self.target_length - len(char_lvl)
        if extra_length>=0:
            char_lvl = char_lvl + ['<pad>']*extra_length
        else:
            char_lvl = char_lvl[:self.target_length -1]+['<eos>']
        #print(f'char lvl: {char_lvl}')
        return self.target_encoder.transform(char_lvl)

    def __getitem__(self,idx):
        image_name = self.dataset.loc[idx,'image_link'].split('/')[-1]
        img_path = self.source_dir+'/'+image_name#os.path.join(self.source_dir,image_name)
        img_array = self.__get_image(img_path)
        #print("image got!")
        entity_name = self.dataset.loc[idx,'entity_name']
        entity_vector = np.array(self.entity_transform.transform([entity_name]),dtype=np.int32)
        entity_value = self.dataset.loc[idx,'entity_value']
        target = self.__create_target(entity_value)
        return {"image":img_array,"question":entity_vector,"target":target}

In [90]:
# '1 gm'
# sos,1," ",g,m,eos,pad,pad,pad,pad,pad,pad,...

In [91]:
unq_l=""
for i in df["entity_value"].unique():
    #print(i)
    for j in i:
        if j not in unq_l:
            unq_l+=j
print(sorted(unq_l))

[' ', '+', ',', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'y']


In [92]:
from sklearn.preprocessing import LabelEncoder
target_chars = sorted(unq_l+'xz')
target_chars=target_chars+['<sos>']+['<eos>']+['<pad>']
target_encoder = LabelEncoder()

In [93]:
target_encoder.fit(target_chars)

LabelEncoder()

In [94]:
question_encoder = LabelEncoder()
question_encoder.fit(df['entity_name'].values)

LabelEncoder()

In [95]:
question_encoder.transform(['height','height']
).shape

(2,)

In [96]:
question_encoder.classes_.shape

(8,)

In [97]:
from sklearn.model_selection import train_test_split

In [98]:
X_train,X_test,Y_train,Y_test = train_test_split(df,df['entity_value'],stratify=df['entity_name'],test_size=0.2)

In [99]:
X_train.reset_index(drop=True,inplace=True)
X_test.reset_index(drop=True,inplace=True)

In [100]:
data_transform=transforms.Compose([
    transforms.Resize((256,256)),
    transforms.ToTensor()
])

In [101]:
train_dataset = CustomDataset(X_train,data_transform,train_dir,35,question_encoder,target_encoder)
val_dataset = CustomDataset(X_test,data_transform,train_dir,35,question_encoder,target_encoder)

In [102]:
(data_transform(PIL.Image.open(train_dir+'/21eHFP5yRTL.jpg')).permute(1, 2, 0).float()/255.0).max()

tensor(0.0039)

In [103]:
next(iter(train_dataset))

{'image': tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],
 
         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],
 
         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]]]),
 'question': array([3], dtype=int32),
 'target': array([16, 13,  5,  3,  4,  0, 28, 27, 29, 32, 25, 35, 19, 30, 14, 15, 15,
        15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 1

In [104]:
train_dataloader = DataLoader(train_dataset,batch_size=64,shuffle=True,drop_last=True)
val_dataloader = DataLoader(val_dataset,batch_size=64,shuffle=False,drop_last=True)

In [105]:
data = next(iter(train_dataloader))

In [106]:
data.keys()

dict_keys(['image', 'question', 'target'])

In [107]:
data['question'].shape

torch.Size([64, 1])

In [108]:
data['image'].shape

torch.Size([64, 3, 256, 256])

In [109]:
import torch
import matplotlib.pyplot as plt

# Create a batch of image tensors (e.g., 4 images, 3 channels, 64x64)
batch_images = data['image']  # (Batch size, Channels, Height, Width)

# Function to display images in a subplot
def imshow_subplot(batch_images):
    num_images = batch_images.size(0)
    print(num_images)
    fig, axs = plt.subplots(1, num_images, figsize=(15, 5))  # Create subplots

    for i in range(num_images):
        image = batch_images[i]
        image = image.permute(1, 2, 0)  # Change shape to (H, W, C)
        image = image.numpy()  # Convert to NumPy array
        
        axs[i].imshow(image)
        axs[i].axis('off')  # Turn off axes for each subplot

    plt.show()

# Display the batch of images
imshow_subplot(batch_images)


64


KeyboardInterrupt: 

In [ ]:
from torchvision import models


In [ ]:
class Encoder(torch.nn.Module):
    def __init__(self,img_dim,total_questions):
        super(Encoder,self).__init__()
        self.img_dim = img_dim
        self.image_model = models.resnet50(pretrained=True)
        self.img_projection = torch.nn.Linear(2048,self.img_dim)
        self.ques_embedding = torch.nn.Embedding(total_questions,128)
        self.image_model.fc=torch.nn.Identity()
        for param in self.image_model.parameters():
            param.requires_grad = False
    
    def forward(self,inputs):
        image_array = inputs['image']
        img = self.image_model(image_array)#batch,128
        img = torch.nn.functional.relu(self.img_projection(img))
        question = inputs['question']
        q = self.ques_embedding(question)#batch,1,128
        q = torch.squeeze(q,1)#batch,128
        return torch.cat((img,q),1)#batch,128+128


In [ ]:
class Decoder(torch.nn.Module):
    def __init__(self,emd_dim,hidden_size,output_chars):
        super(Decoder,self).__init__()
        self.emd_dim = emd_dim
        self.embedding = torch.nn.Embedding(output_chars,self.emd_dim)
        self.lstm = torch.nn.LSTM(self.emd_dim,hidden_size,num_layers=1,batch_first=True)
        self.fc = torch.nn.Linear(hidden_size,output_chars)
    
    def forward(self,inputs,hidden_state,cell):
        x = self.embedding(inputs)
        #print(f'x: {x.shape}')
        x,(hidden,cell) = self.lstm(x,(hidden_state,cell))
        #print(f'from decoder x: {x.shape}')
        output = self.fc(x)
        return output,hidden,cell

In [ ]:
class Final_Model(torch.nn.Module):
    def __init__(self,img_dim,emd_dim,target_length,total_chars,batch_size,device):
        super(Final_Model,self).__init__()
        self.target_length = target_length
        self.device = device
        self.batch_size = batch_size
        self.total_chars = total_chars
        self.encoder = Encoder(img_dim,8)
        self.decoder = Decoder(emd_dim,256,total_chars)
    
    def forward(self,inputs):
        target = inputs['target']
        hidden = cell = self.encoder(inputs) #Batch,256
        hidden = torch.unsqueeze(hidden,0)
        cell = torch.unsqueeze(cell,0)
        final_tokens = torch.zeros(self.target_length,self.batch_size,self.total_chars).to(self.device)
        for i in range(1,self.target_length):
            #target: batch, target_length
            # first token: sos, real taregt, eos,pad.....
            prev_pred = torch.unsqueeze(target[...,i-1],-1)
            curr_pred,hidden,cell = self.decoder(prev_pred,hidden,cell)
            #curr_pred: batch, 1, total_chars
            final_tokens[i] = torch.squeeze(curr_pred,1)
        return final_tokens


In [ ]:
final_model = Final_Model(128,128,35,44,64,device)
final_model = final_model.to(device)

In [ ]:
class AverageLoss(torch.nn.Module):
    def __init__(self,seq_length):
        super(AverageLoss,self).__init__()
        self.seq_length = seq_length
    
    def forward(self,input,target):
        #seq_length,batch,total_chars -> Input
        position_loss = torch.zeros(self.seq_length,dtype=input.dtype)
        for i in range(self.seq_length):
            model_output = input[i,...]#batch,total_chars
            real_target = target[...,i].long()#batch
            position_loss[i] = torch.nn.functional.cross_entropy(model_output,real_target)
        return torch.mean(position_loss)

In [ ]:
loss_fn = torch.jit.script(AverageLoss(35))

In [ ]:
optimizer = torch.optim.Adam(params=final_model.parameters())

In [ ]:
def train_lstm(model:torch.nn.Module,
               dataloader:torch.utils.data.DataLoader,
               loss_fn:torch.nn.Module,
               optimizer:torch.optim.Optimizer):
    model.train()
    train_loss = 0
    for batch,data in tqdm(enumerate(dataloader)):
        X = dict((k,v.to(device)) for k,v in data.items())
        y = data['target'].to(device)
        optimizer.zero_grad()
        y_pred1=model(X)
        loss=loss_fn(y_pred1,y)
        train_loss+=loss.item()

        loss.backward()
        optimizer.step()


    train_loss=train_loss/len(dataloader)
    print(f'train loss: {train_loss}')
    return train_loss

In [ ]:
def test_lstm(model: torch.nn.Module, 
              dataloader: torch.utils.data.DataLoader, 
              loss_fn: torch.nn.Module):
    model.eval() 
    test_loss =0
    with torch.inference_mode():
        for batch, data in tqdm(enumerate(dataloader)):
            X = dict((k,v.to(device)) for k,v in data.items())
            y = data['target'].to(device)
            test_pred_logits = model(X)
            loss = loss_fn(test_pred_logits, y)
            test_loss += loss.item()
             
    test_loss = test_loss / len(dataloader)
    return test_loss

In [111]:

def train(model: torch.nn.Module, 
          train_dataloader: torch.utils.data.DataLoader, 
          test_dataloader: torch.utils.data.DataLoader, 
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module = nn.CrossEntropyLoss(),
          epochs: int = 5):
    
    # 2. Create empty results dictionary
    results = {"train_loss": [],
        "test_loss": [],
    }
    
    # 3. Loop through training and testing steps for a number of epochs
    for epoch in range(epochs):
        train_loss = train_lstm(model=model,
                                           dataloader=train_dataloader,
                                           loss_fn=loss_fn,
                                           optimizer=optimizer)
        test_loss= test_lstm(model=model,
            dataloader=test_dataloader,
            loss_fn=loss_fn)
        
        # 4. Print out what's happening
        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"test_loss: {test_loss:.4f} | "
        )
        torch.save(model.state_dict(),f'saving_model_{epoch}.pt')

        # 5. Update results dictionary
        # Ensure all data is moved to CPU and converted to float for storage
        results["train_loss"].append(train_loss.item() if isinstance(train_loss, torch.Tensor) else train_loss)
        results["test_loss"].append(test_loss.item() if isinstance(test_loss, torch.Tensor) else test_loss)

    # 6. Return the filled results at the end of the epochs
    return results

In [ ]:
train(final_model,train_dataloader,val_dataloader,optimizer=optimizer,loss_fn=loss_fn,epochs=100)

3298it [2:14:54,  2.45s/it]


train loss: 0.325717600847534


824it [32:58,  2.40s/it]


Epoch: 1 | train_loss: 0.3257 | test_loss: 0.3035 | 


1496it [55:53,  2.24s/it]

In [ ]:
import torch

In [ ]:
aaaaaaaaa

In [ ]:
hgchgcj

In [ ]:
torch.load(path_to_state-dict)

In [ ]:
hfchcc